<a href="https://colab.research.google.com/github/SaraElwatany/Text2Pandas-Engine/blob/main/Text2Pandas%20Engine.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Text2Pandas command engine**

Two approaches to construct the Text2Pandas command engine:

**1. First approach:** depends on fully automating the process using llm models like (LLama3, open source model) and Pandasai framework , to perform question answering on our dataframe (**Section 2**)



**2. Second approach:** depends on using llm models like (LLama3, open source model) to extract information in the form of a dictionary, the dictionary will always have the same structure meaning the same keys and only the values of the keys will change based on the parsed data from the llm, and then using the parsed data we would map the values present in it with a predefined python code which will generate the pandas commands (**Section 3**)

In [ ]:
# Required Libraries to install (Assuming you are using Google Colab)
!pip install pandasai
!pip install langchain
!pip install langchain-groq

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.8/179.8 kB 1.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.1/12.1 MB 54.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.5/18.5 MB 48.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 56.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 367.8/367.8 kB 16.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.5/4.5 MB 42.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 411.2/411.2 kB 19.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 334.5/334.5 kB 13.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 318.9/318.9 kB 17.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 4.8 MB/s eta 0:00:00
  Attempting uninstall: sqlglot
    Found existing

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 942.7 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 7.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 396.4/396.4 kB 22.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 290.4/290.4 kB 20.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 141.9/141.9 kB 11.6 MB/s eta 0:00:00
  Attempting uninstall: tenacity
    Found existing installation: tenacity 9.0.0
    Uninstalling tenacity-9.0.0:
      Successfully uninstalled tenacity-9.0.0
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 106.5/106.5 kB 1.8 MB/s eta 0:00:00


In [ ]:
import re
import os
import ast
import sys
import json
import uuid
import logging
import pandas as pd
from pandasai import SmartDataframe
from langchain_groq import ChatGroq
from IPython.display import Markdown, display
from typing import List, Optional, Union, Dict
from langchain.output_parsers import StructuredOutputParser, ResponseSchema
from langchain.prompts import ChatPromptTemplate, FewShotPromptTemplate, PromptTemplate, HumanMessagePromptTemplate
from langchain_core.pydantic_v1 import BaseModel, Field, validator
from langchain_core.messages import (
                                      AIMessage,
                                      BaseMessage,
                                      HumanMessage,
                                      SystemMessage,
                                      ToolMessage,
                                    )

In [ ]:
# Root Directory of the csv file attached for the second task
task_2_dir = 'task2'

### **Section 1. Data Handling & Examination**

In [ ]:
# Read the csv file
task_2_df = pd.read_csv(os.path.join(task_2_dir, 'task2.csv'))

In [ ]:
task_2_df.head()

,cell_no,hour,month,day,year,Bandwidth,counter_0,counter_1,counter_2,counter_3,counter_4,counter_5
0,0,0,6,12,2019,10,470620,367862,214295,3592,123759,"[0.0, 513392.0, 792138.0, 47284.0, 22429.0, 15..."
1,0,1,6,12,2019,10,129657,80729,37125,2698,72283,"[0.0, 1886861.0, 1579023.0, 67836.0, 23810.0, ..."
2,0,2,6,12,2019,10,28839,18523,7949,1,21925,"[0.0, 2024560.0, 1288489.0, 213675.0, 33865.0,..."
3,0,3,6,12,2019,10,17751,17845,5292,0,10683,"[0.0, 2089627.0, 1239282.0, 231054.0, 18927.0,..."
4,0,4,6,12,2019,10,42836,35334,12739,0,24262,"[0.0, 2064013.0, 1355993.0, 132165.0, 18483.0,..."


In [ ]:
# Get the names of the columns associated with that csv file
task_2_df.keys()

Index(['cell_no', 'hour', 'month', 'day', 'year', 'Bandwidth', 'counter_0',
       'counter_1', 'counter_2', 'counter_3', 'counter_4', 'counter_5'],
      dtype='object')

In [ ]:
# Get the datatypes of the columns associated with that csv file
task_2_df.dtypes

,0
cell_no,int64
hour,int64
month,int64
day,int64
year,int64
Bandwidth,int64
counter_0,int64
counter_1,int64
counter_2,int64
counter_3,int64


In [ ]:
# Get the shape of the dataframe
task_2_df.shape

(711432, 12)

In [ ]:
# Check for any null values in that csv file
task_2_df.isnull().sum()

,0
cell_no,0
hour,0
month,0
day,0
year,0
Bandwidth,0
counter_0,0
counter_1,0
counter_2,0
counter_3,0


In [ ]:
# Summary of the csv file
task_2_df.describe()

,cell_no,hour,month,day,year,Bandwidth,counter_0,counter_1,counter_2,counter_3,counter_4
count,711432.000000,711432.000000,711432.0,711432.000000,711432.0,711432.000000,7.114320e+05,7.114320e+05,7.114320e+05,7.114320e+05,7.114320e+05
mean,5052.316264,11.500000,6.0,13.003340,2019.0,13.946632,7.768739e+05,7.296818e+05,2.716741e+05,6.563258e+03,2.678636e+05
std,2917.269491,6.922191,0.0,0.815932,0.0,4.887786,1.310864e+06,1.152211e+06,4.136660e+05,2.190204e+04,4.624766e+05
min,0.000000,0.000000,6.0,12.000000,2019.0,10.000000,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
25%,2529.000000,5.750000,6.0,12.000000,2019.0,10.000000,2.467000e+04,2.563800e+04,1.727100e+04,0.000000e+00,1.483000e+04
50%,5064.000000,11.500000,6.0,13.000000,2019.0,10.000000,2.850315e+05,2.820030e+05,1.038480e+05,3.060000e+02,8.408250e+04
75%,7589.000000,17.250000,6.0,14.000000,2019.0,20.000000,1.076867e+06,1.036334e+06,3.603205e+05,3.685000e+03,3.351872e+05
max,10086.000000,23.000000,6.0,14.000000,2019.0,20.000000,1.105076e+08,1.678519e+08,6.603080e+06,1.307764e+06,1.003652e+07


### **Section 2. First Approach: Using LLama 3 & PandasAI Integration**

In [ ]:
# Function to chat with CSV data
def chat_with_csv(df, query):

    """
    Function to convert the text to pandas query directly using LLM

    Parameters:
    df: The df of the csv file we are interested in.
    query: The provided user query (text).

    Returns:
    result: The response to the query.

    """


    # API Key to be able to use the LLama3 Model
    groq_api_key = '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'

    # Initializing GROQ chat with provided API key, model name, and settings
    llm = ChatGroq(
                    groq_api_key=groq_api_key,
                    model_name="llama3-70b-8192",
                    temperature=0.2)

    # Initialize SmartDataframe with DataFrame and LLM configuration
    pandas_ai = SmartDataframe(df, config={"llm": llm})
    # Chat with the DataFrame using the provided query
    result = pandas_ai.chat(query)

    return result

In [ ]:
def run_engine_first_approach(df):

    """
    Function to run the engine of the first approach

    Parameters:
    df: The df from the csv file we are interested in.

    Returns:
    None

    """

    # Display a message to the user & wait for the user's input
    input_text = input("Enter the query: ")

    # Perform analysis
    if input_text:
            print("Your Query: "+ input_text, "\n")
            result = chat_with_csv(df, input_text)
            # Print the result of the query
            print("Result: " + str(result), "\n")
    else:
            print("No query provided.")

In [ ]:
# Run the cell to try the first approach
run_engine_first_approach(task_2_df)

Enter the query: "What is the maximum value for counter_2 for all of the cells?
Your Query: "What is the maximum value for counter_2 for all of the cells? 

Result: 6603080 



### **Section 3. Second Approach: Using LLama 3 For Query Parsing & Mapping the Parsed Data with the Predefined Pandas Commands**

In [ ]:
def parse_query(query):

  """
    Function to parse the query(user input/text) using LLM (LLama3 specifically)

    Parameters:
    query: The provided user query (text).

    Returns:
    llm_output: This represents the AI Message after parsing the query and following the template we created.

    """


  # Initializing GROQ chat with provided API key, model name, and settings
  groq_api_key = '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'
  llm = ChatGroq(
                  groq_api_key=groq_api_key,
                  model_name="llama3-70b-8192",
                  temperature=0.2)


  # How you would like your response structured.
  response_schemas = [
                      ResponseSchema(name="column", description="The column to perform the operation on"),
                      ResponseSchema(name="filter_condition", description="The filter condition if available, if not available place NULL"),
                      ResponseSchema(name="group_by", description="The group by condition if available for example by cell or column, if not available place NULL"),
                      ResponseSchema(name="aggregation", description="The type of aggregation to perform after grouping either: max, min, mean, sum, count, if not available place NULL"),
                      ResponseSchema(name="descriptive_statistics", description="The basic statistics to perform either: count, mean, median, mode, standard deviation, and percentiles. if not available place NULL"),
                      ResponseSchema(name="summary_statistics", description="The summary statistics of the df either: describe, corr, cov. if not available place NULL"),
                      ResponseSchema(name="unique_condition", description="Either nunique to count the distinct values, unique to return a list of distinct values , or value_countsto shows the frequency of each unique value. If not mentioned place NULL"),

                      ResponseSchema(name="add_column", description="The name of the column we want to add, if not mentioned any addition of columns then place NULL"),
                      ResponseSchema(name="drop_column", description="The name of the column we want to drop, if not mentioned any dropping or removal of columns then place NULL"),

                      ResponseSchema(name="fillna", description="The value/number that fills the missing or NaN values in the df, If not mentioned place NULL"),
                      ResponseSchema(name="dropna", description="The name of the column that contain missing values (NaN) from the df, if not mentioned any sorting then place with NULL"),

                      ResponseSchema(name="sort_order", description="The order to sort by the data either ascending or descinding, if not mentioned any sorting then place with NULL"),
                      ResponseSchema(name="sort_column", description="The column to sort by, if not mentioned any sorting then place with NULL"),
                   ]



  # How you would like to parse your output (Format of the parsed data)
  output_parser = StructuredOutputParser.from_response_schemas(response_schemas)
  format_instructions = output_parser.get_format_instructions()
  # See the prompt template you created for formatting
  #print ('Format Instructions: ', format_instructions)

  # A template for your LLM to work on (can be seen as simply an instructions to your Large Language Model)
  template = f"""
                You will be given a user pandas query in the form of text.
                Extract all the useful information from it to be able to perform the query.
                Keep in mind that the dataframe has the following columns: {str(list(task_2_df.columns))}.
                """ + """
                Use the provided columns to identify if the user query wants to filter based on one of the columns or group the cells based on them.
                When the user pandas query mentions the word for all cells it is talking about the cell_no column, so be careful the group_by condition may not be null.
                The filter condition if available should be in a form for pandas query to understand, such as: "filter_condition": "year > 2019".
                If the query wants to perform operations on several columns for example the median of each feature in the dataset, then make the column in the form of: column_1, column_2, column_3. Where each column represents the features/columns in the dataframe

                Return the output as a valid JSON object, ensuring all commas between key-value pairs are present.

                {format_instructions}

                % USER INPUT:
                {user_input}

                YOUR RESPONSE:
                """


  # Create a prompt template that takes the user input (query/text), the format we want from the output (information retrieved from the query in the form of a json)
  prompt = PromptTemplate(
                          input_variables=["user_input"],
                          partial_variables={"format_instructions": format_instructions},
                          template=template
                         )




  promptValue = prompt.format(user_input=query)
  # print(promptValue)

  # Use the LLM with the instructions made (template) to parse the data
  llm_output = llm.invoke(promptValue)

  # print("Result (AI Message): \n" , llm_output)
  llm_output_text = llm_output.content
  # print("AI Message's Content: \n" , type(llm_output_text), "AI Message's Content: \n" , llm_output_text)

  try:
    json_string = llm_output_text.replace('json', '')

    # Remove the backticks and clean the string
    json_string = json_string.replace('```', '').strip()

    # Further cleanup: Replace 'NULL' with 'null' to make it JSON-compatible
    json_string = json_string.replace('NULL', 'null')

    # Parse the cleaned JSON string
    parsed_output = json.loads(json_string)

    return parsed_output

  except json.JSONDecodeError as e:
      print(f"Error parsing JSON: {e}")
      return None

In [ ]:
# Function to apply operations based on parsed output
def apply_operations(df, parsed_output):

    """
    Function to apply operations(pandas queries) based on the parsed output from the LLM

    Parameters:
    df: The df from the csv file we are interested in.
    parsed_output: The parsed output from the LLM as a dictionary.

    Returns:
    df: The updated dataframe after applying the queries.

    """

    # Dictionary to map operations to pandas commands
    pandas_operations = {

                          # For general descriptive statistics (within a certain column range)
                          "max": lambda df, column: df[column].max(),
                          "min": lambda df, column: df[column].min(),
                          "sum": lambda df, column: df[column].sum(),
                          "mean": lambda df, column: df[column].mean(),
                          "median": lambda df, column: df[column].median(),
                          "mode": lambda df, column: df[column].mode(),
                          "count": lambda df, column: df[column].count(),

                          # For Filtering operation
                          "filter_condition": lambda df, condition: df.query(condition),

                          # For sorting operation
                          "sort": lambda df, column, sort_order: df.sort_values(by=column, ascending=(sort_order == 'ascending')),


                          # For aggregation operations
                          "group_by_sum": lambda df, column, group_by: df.groupby(group_by)[column].sum(),
                          "group_by_min": lambda df, column, group_by: df.groupby(group_by)[column].min(),
                          "group_by_max": lambda df, column, group_by: df.groupby(group_by)[column].max(),
                          "group_by_mean": lambda df, column, group_by: df.groupby(group_by)[column].mean(),
                          "group_by_count": lambda df, column, group_by: df.groupby(group_by)[column].count(),

                          # For Unique Operations
                          "value_counts": lambda df, column: df[column].value_counts(),
                          "nunique": lambda df, column: df[column].nunique(),
                          "unique": lambda df, column: df[column].unique(),


                          # Summary Statistics (within the df range)
                          "describe": lambda df: df.describe(),
                          "corr": lambda df: df.corr(),
                          "cov": lambda df: df.cov(),


                          # Dropping & Adding Columns
                          "add_column": lambda df, column, value: df.assign(**{column: value}),
                          "drop_column": lambda df, column: df.drop(columns=[column]),

                          # Removing Null valued records or filling them with values
                          "fillna": lambda df, column, value: df[column].fillna(value),
                          "dropna": lambda df, column: df.dropna(subset=[column]),
                      }




    # Retrieve the information from the parsed output
    column = parsed_output.get('column')
    filter_condition = parsed_output.get('filter_condition')
    group_by = parsed_output.get('group_by')
    aggregation = parsed_output.get('aggregation')

    summary_statistics = parsed_output.get('summary_statistics')
    descriptive_statistics = parsed_output.get('descriptive_statistics')

    unique_condition = parsed_output.get('unique_condition')

    add_column = parsed_output.get('add_column')
    drop_column = parsed_output.get('drop_column')

    fillna = parsed_output.get('fillna')
    dropna = parsed_output.get('dropna')

    sort_order = parsed_output.get('sort_order')
    sort_column = parsed_output.get('sort_column')


    # Apply filter on df if condition is present
    if filter_condition:
        df = pandas_operations["filter_condition"](df, filter_condition)

    # Apply grouping and aggregation if the conditions are present
    if group_by:
        if aggregation in ["max", "min", "mean", "sum", "count"]:
            df = pandas_operations[f"group_by_{aggregation}"](df, column, group_by)
        else:
            raise ValueError(f"Operation '{aggregation}' not supported with grouping.")

    # If the query doesn't mention grouping then check for basic operations on the individual columns
    else:
        # If the query doesn't mention grouping
        if descriptive_statistics in ["max", "min", "mean", "mode", "median", "sum", "count"]:
            df = pandas_operations[descriptive_statistics](df, column)
        if summary_statistics in ["describe", "cov", "corr"]:
            df = pandas_operations[summary_statistics](df, column)
        if add_column:
            df = pandas_operations["add_column"](df, add_column)
        if drop_column:
            df = pandas_operations["drop_column"](df, drop_column)
        if fillna:
            df = pandas_operations["fillna"](df, column, fillna)
        if dropna:
            df = pandas_operations["dropna"](df, dropna)
        elif unique_condition in ["value_counts", "nunique",  "unique"]:
            df = pandas_operations[unique_condition](df, column)
        # else:
        #     raise ValueError(f"Operation not supported.")

    # If the query mentions sorting
    if sort_order:
        df = pandas_operations['sort'](df, sort_column, sort_order)

    return df

In [ ]:
# Function to apply operations based on parsed output
def run_engine_second_approach(df):

    """
    Function to run the engine of the second approach

    Parameters:
    df: The df from the csv file we are interested in.

    Returns:
    None

    """

    # Display a message to the user & wait for the user's input
    input_text = input("Enter the query: ")
    parsed_output = {}

    # Perform analysis
    if input_text:
            # Pass the user's query to the parsing function & get the AI Message with the parsed data
            parsed_output = parse_query(str(input_text))

            # Print the user's query
            print("Your Query: "+ input_text, "\n")

            # Print the parsed output
            print("The Parsed Data:\n ", parsed_output, "\n")
    else:
            print("No query provided.")

    # Initialize DataFrame
    df_result = df.copy()

    # Execute the operations
    result_df = apply_operations(df_result, parsed_output)

    # print the result of the query
    print('Result of the query: \n', result_df, "\n")

In [ ]:
# Run the cell to try the second approach
run_engine_second_approach(task_2_df)

Enter the query: "What is the maximum value for counter_2 for all of the cells?
Your Query: "What is the maximum value for counter_2 for all of the cells? 

The Parsed Data:
  {'column': 'counter_2', 'filter_condition': None, 'group_by': 'cell_no', 'aggregation': 'max', 'descriptive_statistics': None, 'summary_statistics': None, 'unique_condition': None, 'add_column': None, 'drop_column': None, 'fillna': None, 'dropna': None, 'sort_order': None, 'sort_column': None} 

Result of the query: 
 cell_no
0         552616
1          33622
2         162759
3          17776
4            422
          ...   
10082     890310
10083     239447
10084     882536
10085    1415453
10086    1319282
Name: counter_2, Length: 10041, dtype: int64 



### **Section 4. How to Evaluate & Optimize our engine**

The evaluation of any application is an **iterative** process and it involves several key components which can be divided as follows:

1. An evolving evaluation dataset that is continuously improving over time, the dataset should be of high quality (no noise, no missing or null values, ..etc) , a very important aspect is that it should be representative.

2. Choosing and implementing a set of relevant evaluation metrics like precision, recall, accuracy or even time of response depending on application.

3. Establishing a strong evaluation infrastructure allows for continuous, real-time assessments throughout the entire lifecycle of your LLM application.

In our Case the evaluation of the "Text2PandasEngine" will depend solely on the evaluation of how the input text / user query is parsed, since the quality of the parsed data would affect how the mapping of the commands would work.

Given that the parsing of our data was done by the help of a LLM (specifically LLama3) by incorporating its abilities in the Information Retrieval (IR), our evaluation process would include evaluating its abilities in the IR, and subsequently improving its performance in our application by updating our prompt template to ensure the proper parsing of data.

### **Evaluation of the quality of parsed data**
---



#### **A. Evolving Evaluation Dataset**

An evaluation dataset, also called a golden dataset or ground truth, is crucial for assessing any system's performance. Creating such a dataset involves significant cost and time, as it requires selecting diverse inputs across various scenarios and complexities to ensure the system can generalize well. The golden dataset serves as a benchmark for evaluating the system's capabilities, and identifying improvement areas. There is no strict rule for the number of examples needed, but it's important to cover edge cases. Even a small set of 10-50 examples can be valuable, and additional examples can be added over time to enhance the dataset. For our case we can collect an intial small dataset, the dataset would be in the form of user query, which would represent our input X, and the corresponding expected parsed data (dictionary of values), which would be our target Y, the golden data can be collected from any one for example the data analysts or scientists working in the company. The data can always be updated at anytime so starting with a small amount of data is not a big deal. Another way of collecting data would be the preproduction or predeployment phases, where we can use the engine and provide a section for the evaluation/feedback of the users (the fastest way to collect data, but be careful of the noise).



#### **B. Evaluation Metrics**

After creating our dataset, we can now define some metrics to evaluate our responses on. Since we have an expected answer, we can compare to that as part of our evaluation. Choosing the accuracy, which would represent the amount of correctly parsed data, as an evaluation metric would be a good starting point.



#### **C. Evaluation Infrastructure**

General speaking we could have multiple approaches for that:


**1. Practice of eyeballing:**

The most basic straight forward approach, however not the most reliable if we would to speak on a broader level. This involves experimenting with a few inputs and expected responses, and subsequently improving our system by trying various prompt templates. In general this case would introduce the highest quality output (at the expense of time) since it would depend on the human expertise, however given that our output isn't like a natural language text (our output has a definite structure and values) thus it wouldn't be advised to take that approach.

In our case the Parsed Data should have the same structure (keys) but with different values:

**e.g.**

{ 'column': 'counter_2',
    'filter_condition': None,
    'group_by': 'cell_no',
    'aggregation': 'max',
    'descriptive_statistics': None,
    'summary_statistics': None,
    'unique_condition': None,
    'add_column': None,
    'drop_column': None,
    'fillna': None,
    'dropna': None,
    'sort_order': None,
    'sort_column': None
}

So if the process of comparing the values was automated we would be saving time, human resources (maybe at the expense of computational resources) while also maintaining high quality evaluation.



**2. AI evaluating AI:**

A very common approach, and can be thought of as the optimal approach in our case. This approach would make use of other llms to evaluate the quality of our parsed data by leveraging the power of prompt templates. So how would this work ? well after having our own golden standard dataset we will then use a llm of our choice along with our llm-based engine and using prompts we would instruct the chosen llm to evaluate the parsed data from the pandas engine by giving it both the golden dataset and the output of the pandas engine. There are a lot of frameworks for doing so, for example: LangSmith by LangChain.

**"For more details  =>  https://docs.smith.langchain.com/tutorials/Developers/evaluation "**



**Remember Evaluation is an iterative process , repeat the cycle even after deployment :)**



### **Improving the quality of parsed data**
---

After evaluating the model you can always improve it, and since the evaluation is an iterative process it would be clear that also the system's updating would be iterative (similar to how versions of programs are present). The updating can be based on the data collected from the feedback and then modifying the prompt template given to our application's / engine's llm (LLama3, the one used in the system) to be able to handle new queries & then evaluating again.